In [1]:
# #only palakh
# import os
# os.chdir("D:/Palakh Data/Downloads")
# print(os.getcwd())


In [2]:
# !pip install gensim

In [3]:
import pandas as pd
import numpy as np


import string
import re
from pprint import pprint

# NLTK 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english') #this depends on each language

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
df_depression = pd.read_csv('depression_posts.csv')
print(df_depression.shape)
df_depression.head(3)

(4035, 13)


,title,score,p_id,subreddit,url,num_comments,body,p_timestamp,c_id,comment,c_timestamp,Post_Reply,Time_to_Comment
0,Regular Check-In Post,970,cml6ni,depression,https://www.reddit.com/r/depression/comments/c...,5523,Welcome to /r/depression's check-in post - a p...,2019-08-06 06:12:33,ew4joli,I gave in to my depression and stayed home ins...,2019-08-06 20:21:04,Y,0 days 14:08:31.000000000
1,Regular Check-In Post,970,cml6ni,depression,https://www.reddit.com/r/depression/comments/c...,5523,Welcome to /r/depression's check-in post - a p...,2019-08-06 06:12:33,ewejn04,Currently experiencing: Extreme sadness (but c...,2019-08-09 07:49:31,Y,3 days 01:36:58.000000000
2,Regular Check-In Post,970,cml6ni,depression,https://www.reddit.com/r/depression/comments/c...,5523,Welcome to /r/depression's check-in post - a p...,2019-08-06 06:12:33,ew6orjt,i’m just too scared to kill myself. i wish i c...,2019-08-07 14:34:33,Y,1 days 08:22:00.000000000


In [5]:
data=df_depression['comment'].values.tolist()
data[:10]

["I gave in to my depression and stayed home instead of going to an exam that made up 35% of my grade. Then I thought to myself what the fuck did I just do and I dragged my ass to office hours to beg for another chance and told my professor honestly why I didn't attend. \n\nHe was very sympathetic and told me he could tell I was having a hard time and said I could take it tomorrow. I don't feel like I deserve this chance, but I won't waste it.",
 "Currently experiencing: Extreme sadness (but can't cry), unreasonable bitterness at other people, regret (so many regrets), hatefulness (at myself), suicidal, hopelessness, mentally exhausted.\n\nI really, really hate my existence and I wish I was never born.",
 'i’m just too scared to kill myself. i wish i could die from just closing my eyes.',
 "I feel lonely. I have isolated myself from my friends. I don't talk about anything anymore. What am i going to talk about anyways? I don't want to make them feel shitty. I feel like a failure. I hav

## Tokenize and text cleanup

In [6]:
# Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['gave', 'in', 'to', 'my', 'depression', 'and', 'stayed', 'home', 'instead', 'of', 'going', 'to', 'an', 'exam', 'that', 'made', 'up', 'of', 'my', 'grade', 'then', 'thought', 'to', 'myself', 'what', 'the', 'fuck', 'did', 'just', 'do', 'and', 'dragged', 'my', 'ass', 'to', 'office', 'hours', 'to', 'beg', 'for', 'another', 'chance', 'and', 'told', 'my', 'professor', 'honestly', 'why', 'didn', 'attend', 'he', 'was', 'very', 'sympathetic', 'and', 'told', 'me', 'he', 'could', 'tell', 'was', 'having', 'hard', 'time', 'and', 'said', 'could', 'take', 'it', 'tomorrow', 'don', 'feel', 'like', 'deserve', 'this', 'chance', 'but', 'won', 'waste', 'it']]


## Bigrams & Trigrams

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases. min_count: Ignore all words and bigrams w/ total collected count lower than this value
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['gave', 'in', 'to', 'my', 'depression', 'and', 'stayed', 'home', 'instead', 'of', 'going', 'to', 'an', 'exam', 'that', 'made', 'up', 'of', 'my', 'grade', 'then', 'thought', 'to', 'myself', 'what', 'the', 'fuck', 'did', 'just', 'do', 'and', 'dragged', 'my', 'ass', 'to', 'office', 'hours', 'to', 'beg', 'for', 'another', 'chance', 'and', 'told', 'my', 'professor', 'honestly', 'why', 'didn', 'attend', 'he', 'was', 'very', 'sympathetic', 'and', 'told', 'me', 'he', 'could', 'tell', 'was', 'having', 'hard', 'time', 'and', 'said', 'could', 'take', 'it', 'tomorrow', 'don', 'feel', 'like', 'deserve', 'this', 'chance', 'but', 'won', 'waste', 'it']


### Remove stop words, make bigrams and lemmetize

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

## Let's call the function in order

In [12]:
#To run this code it is necessary to get the spacy model: "!python - m spacy download en", last depends on the used language 

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'es' model, keeping only tagger component (for efficiency)
#python3 -m spacy download es
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv. Returns the base or dictionary form of a word
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['give', 'depression', 'stay', 'home', 'instead', 'go', 'exam', 'make', 'grade', 'think', 'drag', 'ass', 'office', 'hour', 'beg', 'chance', 'tell', 'professor', 'honestly', 'attend', 'sympathetic', 'tell', 'could', 'tell', 'hard', 'time', 'say', 'could', 'take', 'tomorrow', 'feel', 'deserve', 'chance', 'waste']]


## Create the Dictionary and Corpus needed for Topic Modeling

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 3), (26, 1), (27, 1), (28, 1), (29, 1)]]


In [14]:
id2word[0]

'ass'

In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('ass', 1),
  ('attend', 1),
  ('beg', 1),
  ('chance', 2),
  ('could', 2),
  ('depression', 1),
  ('deserve', 1),
  ('drag', 1),
  ('exam', 1),
  ('feel', 1),
  ('give', 1),
  ('go', 1),
  ('grade', 1),
  ('hard', 1),
  ('home', 1),
  ('honestly', 1),
  ('hour', 1),
  ('instead', 1),
  ('make', 1),
  ('office', 1),
  ('professor', 1),
  ('say', 1),
  ('stay', 1),
  ('sympathetic', 1),
  ('take', 1),
  ('tell', 3),
  ('think', 1),
  ('time', 1),
  ('tomorrow', 1),
  ('waste', 1)],
 [('bear', 1),
  ('bitterness', 1),
  ('cry', 1),
  ('currently', 1),
  ('exhaust', 1),
  ('existence', 1),
  ('experience', 1),
  ('extreme', 1),
  ('hate', 1),
  ('hatefulness', 1),
  ('hopelessness', 1),
  ('many', 1),
  ('mentally', 1),
  ('never', 1),
  ('people', 1),
  ('really', 2),
  ('regret', 2),
  ('sadness', 1),
  ('suicidal', 1),
  ('unreasonable', 1),
  ('wish', 1)]]

# Building the Topic Model

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=4,
                                           alpha='auto',
                                           per_word_topics=True)


In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.208*"could" + 0.143*"wish" + 0.116*"reason" + 0.107*"would" + '
  '0.034*"normal" + 0.031*"whole" + 0.025*"cut" + 0.022*"selfish" + '
  '0.018*"dad" + 0.018*"repeat"'),
 (1,
  '0.045*"feel" + 0.032*"go" + 0.031*"know" + 0.031*"get" + 0.026*"think" + '
  '0.025*"life" + 0.024*"time" + 0.023*"want" + 0.022*"really" + '
  '0.021*"thing"'),
 (2,
  '0.109*"understand" + 0.066*"real" + 0.052*"self" + 0.045*"already" + '
  '0.043*"sense" + 0.042*"happiness" + 0.037*"emotion" + 0.036*"push" + '
  '0.033*"next" + 0.030*"depress"'),
 (3,
  '0.076*"fight" + 0.071*"sleep" + 0.061*"run" + 0.048*"moment" + '
  '0.046*"smile" + 0.044*"wake" + 0.040*"alive" + 0.035*"will" + '
  '0.033*"option" + 0.031*"wait"'),
 (4,
  '0.111*"bring" + 0.074*"husband" + 0.060*"drive" + 0.058*"comfort" + '
  '0.055*"book" + 0.053*"delete" + 0.040*"afraid" + 0.033*"honest" + '
  '0.021*"wear" + 0.021*"inside"'),
 (5,
  '0.172*"try" + 0.118*"would" + 0.051*"sometimes" + 0.038*"experience" + '
  '0.033*"happy" + 

### Compute Model Perplexity and Coherence Score

In [18]:
# Compute Perplexity. In information theory, measurement of how well a probability distribution or probability model predicts a sample. Lower is better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score: score that calculates if the words in the same topic make sense when they are put together
# ref: https://towardsdatascience.com/beginners-guide-to-lda-topic-modelling-with-r-e57a5a8e7a25
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.75738461683336

Coherence Score:  0.40843883669553316


### Visualize the topics-keywords 

In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\Hannah\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.412078 -0.142443       1        1  55.755379
13    -0.337479 -0.137190       2        1  12.577526
5     -0.235094  0.302485       3        1   6.615175
6     -0.046944  0.014505       4        1   3.977882
2      0.009830 -0.008142       5        1   2.610291
0     -0.031163  0.268380       6        1   2.263743
19     0.018121 -0.010125       7        1   2.224780
18     0.018232 -0.009324       8        1   2.178668
3      0.034627 -0.023193       9        1   1.912543
17     0.043013 -0.017117      10        1   1.750635
7      0.056142 -0.019824      11        1   1.635978
10     0.050408 -0.018864      12        1   1.591552
4      0.076387 -0.023008      13        1   1.028539
8      0.085865 -0.024007      14        1   0.972430
14     0.096542 -0.025262      15        1   0.901912
11     0.108251 -0.025677      16        1   0.658554
16     0.100422 -0.025292      17        1   0.581157
15     0.110883 -0.025711      18        1   0.552579
12     0.125412 -0.025151      19        1   0.169894
9      0.128624 -0.025040      20        1   0.040781, topic_info=     Category        Freq    Term       Total  loglift  logprob
321   Default  861.000000     try  861.000000  30.0000   30.000
661   Default  773.000000   would  773.000000  29.0000   29.000
365   Default  818.000000    good  818.000000  28.0000   28.000
130   Default  703.000000    much  703.000000  27.0000   27.000
4     Default  356.000000   could  356.000000  26.0000   26.000
...       ...         ...     ...         ...      ...      ...
1907  Topic20    0.005296  hatred    0.954657   2.6103   -8.667
1908  Topic20    0.005296  loathe    0.954655   2.6103   -8.667
1909  Topic20    0.005296    loop    0.954661   2.6103   -8.667
1910  Topic20    0.005296   major    0.954688   2.6102   -8.667
1911  Topic20    0.005296   mourn    0.954653   2.6103   -8.667

[1378 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
770       2  0.983118  absolutely
1024     17  0.945182       abuse
266      12  0.983580      accept
672      13  0.795541     account
273       6  0.918757     achieve
...     ...       ...         ...
151       1  0.990514        year
151       2  0.006723        year
769       1  0.023807         yet
769       9  0.952288         yet
152       3  0.983754       young

[531 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 14, 6, 7, 3, 1, 20, 19, 4, 18, 8, 11, 5, 9, 15, 12, 17, 16, 13, 10])